In [ ]:
import numpy as np
from keras.layers import Conv2D as _Conv2D
from keras.layers import (
    Dense,
    Dropout,
    Flatten,
    Input,
    MaxPooling2D,
    UpSampling2D,
    concatenate,
    LeakyReLU,
    BatchNormalization
)
from keras.models import Model
#from keras.optimizers import Adam
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import backend as K

def Conv2D(filters, size, activation=LeakyReLU(alpha=0.1), kernel_initializer="he_normal"):
    return _Conv2D(
        filters, 
        size, 
        activation=activation, 
        padding="same", 
        kernel_initializer=kernel_initializer,
    )


def get_standard_model(input_size, num_classes):
    
    inputs = keras.Input(shape=input_size + (3,))
    #inputs = Input(input_size)
    
    conv1 = Conv2D(64, 3)(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(64, 3)(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(128, 3)(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(128, 3)(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(256, 3)(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(256, 3)(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3)(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3)(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(1024, 3)(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3)(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(1024, 3)(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = Conv2D(512, 2)(UpSampling2D(size=(2, 2))(conv5))
    merge6 = concatenate([conv4, up6], axis=3)
    conv6 = Conv2D(512, 3)(merge6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(512, 3)(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = Conv2D(256, 2)(UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3)(merge7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(256, 3)(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = Conv2D(128, 2)(UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3)(merge8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(128, 3)(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = Conv2D(64, 2)(UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3)(merge9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(64, 3)(conv9)
    conv9 = BatchNormalization()(conv9)
    
    #conv10 = Conv2D(num_classes, 3)(conv9)
    conv10 = _Conv2D(num_classes, 1, activation="softmax")(conv9)

    model = Model(inputs=inputs, outputs=conv10, name='Unet')

    return model


In [ ]:
#ids = ['684913']

#MISSION
ids_MISSION = ['684913',
    '684914',
    '684916',
    '685039',
    '685040',
    '685041',
    '685042',
    '685281']
#BEHOLD
ids_BEHOLD = ['688058',
    '688059',
    '688060',
    '688061',
    '688062',
    '688063',
    '688064',
    '688065',
    '688066',
    '688067',
    '688068',
    '688069',
    '688070',
    '688071',
    '688072',
    '688073',
    '688074',
    '688075',
    '688076',
    '688077',
    '688078',
    '688079',
    '688080',
    '688081',
    '688082',
    '688083',
    '688084',
    #'688085',
    '688135']

In [ ]:
from scipy import ndimage
from skimage.segmentation import watershed, random_walker
from skimage.feature import peak_local_max
import tensorflow as tf
from tensorflow.compat.v1.nn import softmax_cross_entropy_with_logits_v2
import matplotlib.pyplot as plt
import skimage.io
from numpy import zeros
import math
from scipy.ndimage.morphology import distance_transform_edt
import glob, os
#
import cv2
import numpy as np
#from PIL import Image
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,10000).__str__()

from PIL import Image
#Image.MAX_IMAGE_PIXELS = 1000000000 
Image.MAX_IMAGE_PIXELS = None

img_size_x = 512
img_size_y = 512
img_size = (img_size_x, img_size_y)
num_classes = 10

weights = np.ones((512,512))
weights[256,256] = 0
weights[255,256] = 0
weights[256,255] = 0
weights[256,255] = 0
distance_map = distance_transform_edt(weights)
max_value = np.amax(distance_map)
distance_map = distance_map / (max_value+1)
distance_map = (1-distance_map)
#print(distance_map.shape)

distance_map2 = np.expand_dims(distance_map, axis=0)
distance_map2 = np.repeat(distance_map2[:,:], 10, axis=0)
# print(distance_map2.shape)


# import matplotlib.pyplot as plt
# plt.rcParams["figure.figsize"] = (25,25)
# fig = plt.figure()
# ax1 = fig.add_subplot(5,5,1)
# ax1.imshow(distance_map2[9,:,:])

#model = keras.models.load_model("standard.h5", compile=True, options=None, custom_objects={ 'softmax_cross_entropy_with_logits_v2': softmax_cross_entropy_with_logits_v2, 'soft_dice_loss': soft_dice_loss, 'binary_crossentropy_Unet': binary_crossentropy_Unet, 'LeakyReLU': LeakyReLU})
model = get_standard_model(img_size, num_classes)
#model.load_weights('chk_BEHOLD/model050.h5')
#model.load_weights('chk_ALL/model050.h5')
model.load_weights('chk/model050.h5')

#directory_out = './Inference/'
#directory_out = "../../Storage/Inference/"
#directory_out = "../../Storage/BL162410_4/"
#directory_out = "illustration_regions/"
directory_out = "ExternalData/"


#os.chdir('GitRepositories/Vessel/masks/')
#for file in glob.glob("./masks/*.jpg"):
#for file in glob.glob("../../Storage/BL162410_4/*.tif"):
for file in glob.glob("ExternalData/*.tif"):
    
    #if any(substring in file for substring in ids_MISSION):
    if True:

        #print(file)
        filename = os.path.basename(file[:-4])
        print(filename)
        patientID = filename.split("_")
        #print(patientID[0])


        #img = cv2.imread("masks/684913_(1.00,2533,15395,1024,2560).jpg")   
        #img = skimage.io.imread("masks/684913_(1.00,2533,15395,1024,2560).jpg")  
        img_original = skimage.io.imread(file)  
        #img = Image.fromarray(img, "RGB")
        #img = np.array(img)

        # padding = 512
        # stepsize = 512//4
        # border = 0
        
        padding = 512
        stepsize = 384
        border = 0

        img = np.pad(img_original, ((padding,padding),(padding,padding),(0,0)), mode='reflect')
        height, width = img.shape[:2]

        #img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_map = zeros([10,height,width])
        img_map_count = zeros([height,width])


        x_range = math.floor(width/stepsize)
        y_range = math.floor(height/stepsize)


        #for g, fn in enumerate(in_file):



        #print("predicting")
        for i in range(0, x_range):
            for j in range(0, y_range):

                x = i*stepsize
                y = j*stepsize
                h = 512
                w = 512
                img_cropped_orig = img[y:y+h, x:x+w]

                if img_cropped_orig.shape[0] == h and img_cropped_orig.shape[1] == w:

                    img_cropped_orig = np.expand_dims(img_cropped_orig, axis=0)
                    mask_pred_orig = model(img_cropped_orig)
                    mask_pred_orig = mask_pred_orig[0,:,:,:]
                    mask_pred_orig = np.transpose(mask_pred_orig, (2, 0, 1))

                    #print(mask_pred_orig.shape)

                    #mask_pred_orig = mask_pred.squeeze().cpu().detach().numpy()
                    #img_map[:,y+border:y+h-border, x+border:x+w-border] += mask_pred_orig[:,0+border:h-border, 0+border:w-border]
                    
                    #img_map[:,y:y+h, x:x+w] += mask_pred_orig * distance_map2
                    img_map[:,y+border:y+h-border, x+border:x+w-border] += mask_pred_orig[:,0+border:h-border, 0+border:w-border] * distance_map2[:,0+border:h-border, 0+border:w-border]

                    #ones_matrix = ones([512,512])
                    #img_map_count[y+border:y+h-border, x+border:x+w-border] += ones_matrix[0+border:h-border, 0+border:w-border]
                    
                    #img_map_count[y:y+h, x:x+w] += distance_map
                    img_map_count[y+border:y+h-border, x+border:x+w-border] += distance_map[0+border:h-border, 0+border:w-border]


        img_map_count = img_map_count[padding:height-padding, padding:width-padding]
        img = img[padding:height-padding, padding:width-padding]
        img_map = img_map[:,padding:height-padding, padding:width-padding] / img_map_count[:,:]

        if True:
            img_map[0,:,:] = ndimage.gaussian_filter(img_map[0,:,:], sigma=5)
            img_map[1,:,:] = ndimage.gaussian_filter(img_map[1,:,:], sigma=5)
            img_map[2,:,:] = ndimage.gaussian_filter(img_map[2,:,:], sigma=5)
            img_map[3,:,:] = ndimage.gaussian_filter(img_map[3,:,:], sigma=5)
            #img_map[4,:,:] = ndimage.gaussian_filter(img_map[4,:,:], sigma=2)
            img_map[5,:,:] = ndimage.gaussian_filter(img_map[5,:,:], sigma=5)
            img_map[6,:,:] = ndimage.gaussian_filter(img_map[6,:,:], sigma=5)
            img_map[7,:,:] = ndimage.gaussian_filter(img_map[7,:,:], sigma=5)
            #img_map[8,:,:] = ndimage.gaussian_filter(img_map[8,:,:], sigma=2)
            img_map[9,:,:] = ndimage.gaussian_filter(img_map[9,:,:], sigma=5)

        mask_pred = np.argmax(img_map, axis=0)

        mask_pred_tumor = (mask_pred == 7).astype(int)
        mask_pred_benign = (mask_pred == 2).astype(int)
        mask_pred_adipose = (mask_pred == 0).astype(int)
        mask_pred_background = (mask_pred == 1).astype(int)
        mask_pred_lymphocytes = (mask_pred == 3).astype(int)
        mask_pred_macrophages = (mask_pred == 4).astype(int)
        mask_pred_muscle = (mask_pred == 5).astype(int)
        mask_pred_nerve = (mask_pred == 6).astype(int)
        mask_pred_vessel = (mask_pred == 8).astype(int)
        mask_pred_stroma = (mask_pred == 9).astype(int)


        def remove_small_regions(m_mask, m_img, threshold): 
            nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(m_mask.astype(np.uint8), connectivity=4)
            sizes = stats[1:, -1];
            for i in range(0, nb_components - 1):
                if sizes[i] < threshold:
                    m_img[output == i + 1] = 1

        if True:
            print('remove small regions in background')
            mask_pred_background_inverse = (1-mask_pred_background)
            remove_small_regions(mask_pred_background_inverse, mask_pred_background, 10000)

            mask_pred_tumor = mask_pred_tumor > mask_pred_background
            mask_pred_benign = mask_pred_benign > mask_pred_background
            mask_pred_adipose = mask_pred_adipose > mask_pred_background
            mask_pred_lymphocytes = mask_pred_lymphocytes > mask_pred_background
            mask_pred_macrophages = mask_pred_macrophages > mask_pred_background
            mask_pred_muscle = mask_pred_muscle > mask_pred_background
            mask_pred_nerve = mask_pred_nerve > mask_pred_background
            mask_pred_vessel = mask_pred_vessel > mask_pred_background
            mask_pred_stroma = mask_pred_stroma > mask_pred_background


            print('remove small regions in stroma')
            mask_pred_stroma_inverse = (1-mask_pred_stroma)
            remove_small_regions(mask_pred_stroma_inverse, mask_pred_stroma, 1000)

            mask_pred_tumor = mask_pred_tumor > mask_pred_stroma
            mask_pred_benign = mask_pred_benign > mask_pred_stroma
            mask_pred_adipose = mask_pred_adipose > mask_pred_stroma
            mask_pred_background = mask_pred_background > mask_pred_stroma
            mask_pred_lymphocytes = mask_pred_lymphocytes > mask_pred_stroma
            mask_pred_muscle = mask_pred_muscle > mask_pred_stroma
            mask_pred_nerve = mask_pred_nerve > mask_pred_stroma

            print('remove small regions in adipose')
            mask_pred_adipose_inverse = (1-mask_pred_adipose)
            remove_small_regions(mask_pred_adipose_inverse, mask_pred_adipose, 10000)

            mask_pred_tumor = mask_pred_tumor > mask_pred_adipose
            mask_pred_benign = mask_pred_benign > mask_pred_adipose
            mask_pred_background = mask_pred_background > mask_pred_adipose
            mask_pred_lymphocytes = mask_pred_lymphocytes > mask_pred_adipose
            mask_pred_muscle = mask_pred_muscle > mask_pred_adipose
            mask_pred_nerve = mask_pred_nerve > mask_pred_adipose
            mask_pred_stroma = mask_pred_stroma > mask_pred_adipose


            print('recover macrophages')
            mask_pred_tumor = mask_pred_tumor > mask_pred_macrophages
            mask_pred_benign = mask_pred_benign > mask_pred_macrophages
            mask_pred_adipose = mask_pred_adipose > mask_pred_macrophages
            mask_pred_background = mask_pred_background > mask_pred_macrophages
            mask_pred_lymphocytes = mask_pred_lymphocytes > mask_pred_macrophages
            mask_pred_muscle = mask_pred_muscle > mask_pred_macrophages
            mask_pred_nerve = mask_pred_nerve > mask_pred_macrophages
            mask_pred_stroma = mask_pred_stroma > mask_pred_macrophages

        if True:
            print('fill vessel holes with borders')
            #mask_pred_vessel = ndimage.binary_fill_holes(mask_pred_vessel)

            mask_pred_vessel = mask_pred_vessel.astype(np.uint8)
            h, w = mask_pred_vessel.shape[:2]

            mask_pred_vessel = cv2.copyMakeBorder(mask_pred_vessel,100,100,0,0,cv2.BORDER_CONSTANT,value=1) # top bottom left right
            mask_pred_vessel = ndimage.binary_fill_holes(mask_pred_vessel)
            mask_pred_vessel = mask_pred_vessel[100:100+h, 0:w]
            mask_pred_vessel = np.array(mask_pred_vessel, dtype=np.uint8)

            mask_pred_vessel = cv2.copyMakeBorder(mask_pred_vessel,0,0,100,100,cv2.BORDER_CONSTANT,value=1) # top bottom left right
            mask_pred_vessel = ndimage.binary_fill_holes(mask_pred_vessel)
            mask_pred_vessel = mask_pred_vessel[0:h, 100:100+w]
            mask_pred_vessel = np.array(mask_pred_vessel, dtype=np.uint8)

            mask_pred_tumor = mask_pred_tumor > mask_pred_vessel
            mask_pred_benign = mask_pred_benign > mask_pred_vessel
            mask_pred_adipose = mask_pred_adipose > mask_pred_vessel
            mask_pred_background = mask_pred_background > mask_pred_vessel
            mask_pred_lymphocytes = mask_pred_lymphocytes > mask_pred_vessel
            mask_pred_macrophages = mask_pred_macrophages > mask_pred_vessel
            mask_pred_muscle = mask_pred_muscle > mask_pred_vessel
            mask_pred_nerve = mask_pred_nerve > mask_pred_vessel
            mask_pred_stroma = mask_pred_stroma > mask_pred_vessel



        if False:
            print("distinguishing tumor/benign")
            mask_pred_tumor_benign = np.maximum(mask_pred_tumor, mask_pred_benign)
            nlabels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask_pred_tumor_benign.astype(np.uint8), connectivity=8)

            lookup_table = np.zeros((nlabels, 2)).astype(float)
            height, width = mask_pred_tumor_benign.shape[:2]

            for x in range(0, width):
                for y in range(0, height):
                    i = labels[y,x]
                    lookup_table[i][0] += img_map[7,:,:][y,x]
                    lookup_table[i][1] += img_map[2,:,:][y,x]


            for x in range(0, width):
                for y in range(0, height):
                    i = labels[y,x]
                    if i > 0:
                        if (lookup_table[i][0] / lookup_table[i][1]) >= 0.5:
                            mask_pred_tumor[y,x] = 1
                            mask_pred_benign[y,x] = 0
                        else:
                            mask_pred_tumor[y,x] = 0
                            mask_pred_benign[y,x] = 1
                    else:
                        mask_pred_tumor[y,x] = 0
                        mask_pred_benign[y,x] = 0



        if True:
            cv2.imwrite(directory_out + filename + '_Tumor.png', 255*mask_pred_tumor)
            cv2.imwrite(directory_out + filename + '_Benign.png', 255*mask_pred_benign)
            cv2.imwrite(directory_out + filename + '_Adipose.png', 255*mask_pred_adipose)
            cv2.imwrite(directory_out + filename + '_Background.png', 255*mask_pred_background)
            cv2.imwrite(directory_out + filename + '_Lymphocytes.png', 255*mask_pred_lymphocytes)
            cv2.imwrite(directory_out + filename + '_Macrophages.png', 255*mask_pred_macrophages)
            cv2.imwrite(directory_out + filename + '_Muscle.png', 255*mask_pred_muscle)
            cv2.imwrite(directory_out + filename + '_Nerve.png', 255*mask_pred_nerve)
            cv2.imwrite(directory_out + filename + '_Stroma.png', 255*mask_pred_stroma)
            cv2.imwrite(directory_out + filename + '_Vessel.png', 255*mask_pred_vessel)


        height, width = mask_pred_vessel.shape[:2]

        rgbArray = np.zeros((height, width,3), 'uint8')
        rgbArray[..., 0] = 255*mask_pred_benign[:,:]
        rgbArray[..., 1] = 255*mask_pred_vessel[:,:]
        rgbArray[..., 2] = 255*mask_pred_tumor[:,:]

        rgbArray2 = np.zeros((height, width,3), 'uint8')
        rgbArray2[..., 0] = 150*mask_pred_stroma[:,:]
        rgbArray2[..., 1] = 150*mask_pred_stroma[:,:]
        rgbArray2[..., 2] = 150*mask_pred_stroma[:,:]

        rgbArray3 = np.zeros((height, width,3), 'uint8')
        rgbArray3[..., 0] = 163*mask_pred_adipose[:,:]
        rgbArray3[..., 1] = 244*mask_pred_adipose[:,:]
        rgbArray3[..., 2] = 252*mask_pred_adipose[:,:]

        rgbArray4 = np.zeros((height, width,3), 'uint8')
        rgbArray4[..., 0] = 170*mask_pred_lymphocytes[:,:]
        rgbArray4[..., 1] = 0*mask_pred_lymphocytes[:,:]
        rgbArray4[..., 2] = 100*mask_pred_lymphocytes[:,:]

        rgbArray5 = np.zeros((height, width,3), 'uint8')
        rgbArray5[..., 0] = 0*mask_pred_macrophages[:,:]
        rgbArray5[..., 1] = 100*mask_pred_macrophages[:,:]
        rgbArray5[..., 2] = 255*mask_pred_macrophages[:,:]

        rgbArray6 = np.zeros((height, width,3), 'uint8')
        rgbArray6[..., 0] = 120*mask_pred_muscle[:,:]
        rgbArray6[..., 1] = 120*mask_pred_muscle[:,:]
        rgbArray6[..., 2] = 50*mask_pred_muscle[:,:]

        rgbArray7 = np.zeros((height, width,3), 'uint8')
        rgbArray7[..., 0] = 190*mask_pred_nerve[:,:]
        rgbArray7[..., 1] = 110*mask_pred_nerve[:,:]
        rgbArray7[..., 2] = 255*mask_pred_nerve[:,:]


        rgbArray = np.maximum(rgbArray, np.maximum(rgbArray2, np.maximum(rgbArray3, np.maximum(rgbArray4, np.maximum(rgbArray5, np.maximum(rgbArray6, rgbArray7))))))

        #cv2.imwrite('combined.bmp', rgbArray)
        cv2.imwrite(directory_out + filename + '_Combined.png', rgbArray)

        rgbArray = cv2.cvtColor(rgbArray, cv2.COLOR_RGB2BGR)
        
        
        plt.rcParams["figure.figsize"] = (20,20)
        fig = plt.figure()
        ax1 = fig.add_subplot(1,2,1)
        ax1.imshow(img_original)
        ax1.axes.get_xaxis().set_visible(False)
        ax1.axes.get_yaxis().set_visible(False)
        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(rgbArray)
        ax2.axes.get_xaxis().set_visible(False)
        ax2.axes.get_yaxis().set_visible(False)
        plt.show()
        